# Improved Version

In [2]:
import gzip
import csv
from collections import defaultdict
import random
import numpy
import pandas
import scipy
import scipy.optimize
from sklearn.linear_model import LogisticRegression

In [3]:
def readGz(path):
    for l in gzip.open(path, "rt"):
        yield eval(l)

In [4]:
def readCSV(path):
    f = gzip.open(path, "rt")
    f.readline()
    for l in f:
        yield l.strip().split(",")

## Reading Prediction

In [150]:
dataset = list(readCSV("train_Interactions.csv.gz"))

In [151]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [152]:
training = dataset[:190000]
validation = dataset[190000:]

In [153]:
# create empty dictionaries of booksPerUser and UsersPerBook
booksPerUser = defaultdict(set)
usersPerBook = defaultdict(set)

# create an empty set of books
booklist = set()

In [154]:
# books and users information in the whole set
for u, b, r in dataset:
    booksPerUser[u].add(b)
    usersPerBook[b].add(u)

    if b not in booklist:
        booklist.add(b)

In [86]:
len(booklist)

7170

In [155]:
# books and users information in training set
booksPerUser_t = defaultdict(set)
usersPerBook_t = defaultdict(set)

# create an empty set of books
booklist_t = set()

for u, b, r in training:
    booksPerUser_t[u].add(b)
    usersPerBook_t[b].add(u)

In [156]:
# append 10000 data (notRead) to validation set
for u, b, r in validation[0:10000]:
    notRead = booklist.difference(booksPerUser[u])
    c = random.sample(notRead, 1)
    line = [u, c[0], '0']
    validation.append(line)

In [157]:
random.shuffle(validation)

In [158]:
new_training = validation[:18000]
new_validation = validation[18000:]

In [159]:
#bookCount_new_train = defaultdict(int)
#new_train_Read = 0

# for user,book,_ in readCSV("train_Interactions.csv.gz"):
#for u, b, r in training:
 #   bookCount[b] += 1
  #  totalRead += 1

#mostPopular = [(bookCount[x], x) for x in bookCount]
#mostPopular.sort()
#mostPopular.reverse()

In [183]:
# similarity in new_training set
new_train_sim = []
for u, b, r in new_training:
    #max_sim = 0
    #mean_sim = 0
    sim_l = []

    # users who have read this book
    users = usersPerBook_t[b]

    # loop through every book in the training set that the user has read
    for b2 in booksPerUser_t[u]:  # for all books in the training set
        if b == b2:
            continue
        sim = Jaccard(users, usersPerBook_t[b2])
        sim_l.append(sim)
        #if sim > max_sim:
            #max_sim = sim
        #mean_sim = mean_sim + sim
    #mean_sim = mean_sim/len(booksPerUser_t[u])
    new_train_sim.append(numpy.mean(sim_l))
    #new_train_sim.append(mean_sim)

TypeError: unsupported operand type(s) for *: 'set' and 'set'

In [161]:
# get number of people who read each book in new_training
new_train_count = []

for u, b, r in new_training:
    num = len(usersPerBook_t[b])/(0.3*7169)
    new_train_count.append(num)

In [162]:
new_train_score = [values[2] for values in new_training]

new_train_label = []
for i in new_train_score:
    if i == '0':
        new_train_label.append(0)
    else:
        new_train_label.append(1)

In [163]:
df = {"new_train_sim": new_train_sim, "new_train_count": new_train_count, "offset": 1}
table = pandas.DataFrame(df, columns=["new_train_sim", "new_train_count", "offset"])
table

,new_train_sim,new_train_count,offset
0,0.010582,0.076719,1
1,0.006579,0.027898,1
2,0.035714,0.009764,1
3,0.029412,0.026503,1
4,0.013986,0.175757,1
...,...,...,...
17995,0.020080,0.205050,1
17996,0.008264,0.133910,1
17997,0.016129,0.027898,1
17998,0.013514,0.032083,1


In [164]:
X_new_train = table.values
y_new_train = new_train_label

In [165]:
model = LogisticRegression(C=10000,class_weight={0:0.46, 1:0.54},solver="lbfgs",max_iter=8000)

In [166]:
model.fit(X_new_train, y_new_train)

LogisticRegression(C=10000, class_weight={0: 0.46, 1: 0.54}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=8000, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [167]:
# similarity in new_validation set
new_valid_sim = []
for u, b, r in new_validation:
    #max_sim = 0
    #mean_sim = 0
    sim_l = []

    # users who have read this book
    users = usersPerBook_t[b]

    # loop through every book in the training set that the user has read
    for b2 in booksPerUser_t[u]:  # for all books in the training set
        if b == b2:
            continue
        sim = Jaccard(users, usersPerBook_t[b2])
        sim_l.append(sim)
       # mean_sim = mean_sim + sim
    #mean_sim = mean_sim/len(booksPerUser_t[u])
        #if sim > max_sim:
            #max_sim = sim
    new_valid_sim.append(numpy.max(sim_l))
    #new_valid_sim.append(mean_sim)

In [168]:
# get number of people who read each book in new_validation
new_valid_count = []

for u, b, r in new_validation:
    num = len(usersPerBook_t[b])/(0.3*7169)
    new_valid_count.append(num)

In [169]:
new_valid_score = [values[2] for values in new_validation]

new_valid_label = []
for i in new_valid_score:
    if i == '0':
        new_valid_label.append(0)
    else:
        new_valid_label.append(1)

In [170]:
df2 = {"new_valid_sim": new_valid_sim, "new_valid_count": new_valid_count, "offset": 1}
table2 = pandas.DataFrame(df2, columns=["new_valid_sim", "new_valid_count", "offset"])
table2

,new_valid_sim,new_valid_count,offset
0,0.008696,0.046032,1
1,0.021277,0.143674,1
2,0.008621,0.531455,1
3,0.017857,0.122751,1
4,0.000000,0.012554,1
...,...,...,...
1995,0.014706,0.015344,1
1996,0.026316,0.019529,1
1997,0.008850,0.020923,1
1998,0.017241,0.013949,1


In [171]:
X_new_valid = table2.values

In [172]:
pred_new_valid = model.predict(X_new_valid)

In [173]:
corrPred_new_valid = pred_new_valid == new_valid_label

In [174]:
sum(corrPred_new_valid) / len(corrPred_new_valid)

0.661

In [175]:
similarity = []
count = []

header = "userID-bookID,prediction" + "\n"

data = open("pairs_Read.txt", "r")
for l in data:
    if l.startswith("userID-bookID,prediction"):
        continue
    else:
        u, b = l.strip().split("-")

    #mean_sim = 0
    #max_sim = 0
    sim_l = []

    # users who have read this book
    users = usersPerBook[b]

    # loop through every book in the training set that the user has read
    for b2 in booksPerUser[u]:  # for all books in the training set
        if b == b2:
            continue
        sim = Jaccard(users, usersPerBook[b2])
        sim_l.append(sim)
        #if sim > max_sim:
                #max_sim = sim
        #mean_sim = mean_sim + sim
    #mean_sim = mean_sim/len(booksPerUser[u])
    #similarity.append(mean_sim)
    similarity.append(numpy.max(sim))
    
    num = len(usersPerBook[b])/(0.3*len(booklist))
    count.append(num)

In [176]:
df3 = {"test_sim": similarity, "test_count": count, "offset": 1}
table3 = pandas.DataFrame(df3, columns=["test_sim", "test_count", "offset"])
table3

,test_sim,test_count,offset
0,0.000000,0.025105,1
1,0.000000,0.016736,1
2,0.000000,0.041841,1
3,0.023256,0.052999,1
4,0.009524,0.114365,1
...,...,...,...
19995,0.006211,0.023710,1
19996,0.000000,0.252441,1
19997,0.000000,0.018131,1
19998,0.000000,0.013947,1


In [177]:
X_test = table3.values

In [178]:
pred_test = model.predict(X_test)

In [181]:
sum(pred_test)

4379

In [180]:
count = 0
header = "userID-bookID,prediction" + "\n"

pred = open("predictions_Read_optimize.txt", "w")
pred.write(header)

data = open("pairs_Read.txt", "r")

for l in data:
    if l.startswith("userID-bookID,prediction"):
        continue
    else:
        u, b = l.strip().split("-")
    
    pred.write(u + "-" + b + "," + str(pred_test[count]) + "\n")
    count = count + 1

pred.close()

In [249]:
count

10000

In [14]:
bookCount = defaultdict(int)
totalRead = 0

# for user,book,_ in readCSV("train_Interactions.csv.gz"):
for u, b, r in training:
    bookCount[b] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

In [16]:
bookCount

defaultdict(int,
            {'b14275065': 105,
             'b82152306': 97,
             'b44882292': 40,
             'b79927466': 19,
             'b05683889': 123,
             'b86375465': 16,
             'b92838791': 37,
             'b35165110': 17,
             'b17128180': 57,
             'b34596567': 21,
             'b23506537': 18,
             'b95176169': 71,
             'b03231083': 16,
             'b20090710': 60,
             'b62660670': 38,
             'b01071719': 44,
             'b99816042': 69,
             'b79202035': 19,
             'b39996061': 59,
             'b15891222': 20,
             'b01287078': 37,
             'b19581977': 19,
             'b80735440': 71,
             'b29820018': 25,
             'b57056376': 29,
             'b50835324': 83,
             'b46969785': 62,
             'b30423629': 43,
             'b63987437': 77,
             'b88993982': 23,
             'b71971959': 9,
             'b11333881': 21,
             'b6018770

In [ ]:
random.shuffle(validation)

## Rating Prediction

In [5]:
dataset = list(readCSV("train_Interactions.csv.gz"))

In [6]:
usersPerBook = defaultdict(dict)
booksPerUser = defaultdict(dict)
for u,b,r in dataset:
    usersPerBook[b][u] = r
    booksPerUser[u][b] = r

In [7]:
# get alpha, beta_u and beta_i when lambda is 3
beta_u_best = {}
beta_b_best = {}

for user_id, book_id, rating in dataset:
    beta_u_best[user_id] = 0
    beta_b_best[book_id] = 0
    
for i in range(1, 100):
    
    # loop through the training set to calculate alpha
    sum_a = 0
    for user_id, book_id, rating in dataset:
        sum_a = sum_a + float(rating) - float(beta_u_best[user_id]) - float(beta_b_best[book_id])
    alpha_best = sum_a/len(dataset)
    
        # loop through the user list
    for u in booksPerUser:
        # loop though book that this user has rated in usersPerBook_t and calculate beta_u
        sum_u = 0
        for b in booksPerUser[u]:
            sum_u = sum_u + float(booksPerUser[u][b]) - alpha_best - float(beta_b_best[b])
        beta_u = sum_u/(2+len(booksPerUser[u]))
        beta_u_best[u] = beta_u
    
    # loop through the book list
    for b in usersPerBook:
        # loop though every user that has rated this book in booksPerUser_t and calculate beta_b
        sum_b = 0
        for u in usersPerBook[b]:
            sum_b = sum_b + float(usersPerBook[b][u]) - alpha_best - float(beta_u_best[u])
        beta_b = sum_b/(18+len(usersPerBook[b]))
        beta_b_best[b] = beta_b

In [8]:
users = list()
books = list()
ratings = list()
count = 0

# get user_id and book_id from file
data = open("pairs_Rating.txt", "r")
for l in data:
    if l.startswith("userID-bookID,prediction"):
        continue
    # get user_id and book_id
    else:
        user_id, book_id = l.strip().split("-")
        count = count + 1
    
    # predict rating
    if user_id in beta_u_best:
        if book_id in beta_b_best:
            pred = alpha_best + float(beta_u_best[user_id]) + float(beta_b_best[book_id])
        else:
            pred = alpha_best + float(beta_u_new[user_id])
    else:
        if book_id in beta_b_best:
            pred = alpha_best + float(beta_u_best[user_id])
        else:
            pred = alpha_best
    
    users.append(user_id)
    books.append(book_id)
    ratings.append(pred)
    
# use our model to do prediction
predictions9 = open("predictions_Rating.txt", "w")
header = "userID-bookID,prediction" + "\n"
predictions9.write(header)

for i in range(0, count):
    predictions9.write(users[i] + "-" + books[i] + "," + str(ratings[i]) + "\n")

predictions9.close()